# Lib

In [25]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import glob

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")


from sklearn.model_selection import train_test_split
from utils import utils_ml, parameters


from ipywidgets import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

18/11/2022  19:16    <DIR>          .
18/11/2022  16:18    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
21/11/2022  23:10         1.006.890 1.EA.ipynb
18/11/2022  20:25            67.359 2.Feature_engineering.ipynb
19/11/2022  02:54            22.885 3.Stability_var.ipynb
21/11/2022  22:52           128.728 4.Modeling.ipynb
21/11/2022  19:26    <DIR>          catboost_info
               4 arquivo(s)      1.225.862 bytes
               4 pasta(s)   340.905.963.520 bytes dispon�veis


# Reading

In [3]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [31]:
if sys.platform=='win32':
    df = pd.read_csv('.\..\data\dados.csv', sep=',', decimal=".")
        
elif sys.platform=='linux':
    df = pd.read_csv('./../data/dados.csv', sep=',', decimal=".")


## dropping column "o" becaise it has 78% missing. and this is a boolean columns
# df.drop("o", axis=1, inplace=True)

In [32]:
df

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,fecha,monto,score,fraude
0,4,0.7685,94436.24,20.0,0.444828,1.0,BR,5,Máquininha Corta Barba Cabelo Peito Perna Pelo...,cat_8d714cd,0.883598,240.0,102.0,1,NaN,N,2020-03-27 11:51:16,5.64,66,0
1,4,0.7550,9258.50,1.0,0.000000,33.0,BR,0,Avental Descartavel Manga Longa - 50 Un. Tnt ...,cat_64b574b,0.376019,4008.0,0.0,1,Y,N,2020-04-15 19:58:08,124.71,72,0
2,4,0.7455,242549.09,3.0,0.000000,19.0,AR,23,Bicicleta Mountain Fire Bird Rodado 29 Alumini...,cat_e9110c5,0.516368,1779.0,77.0,1,NaN,N,2020-03-25 18:13:38,339.32,95,0
3,4,0.7631,18923.90,50.0,0.482385,18.0,BR,23,Caneta Delineador Carimbo Olho Gatinho Longo 2...,cat_d06e653,0.154036,1704.0,1147.0,1,NaN,Y,2020-04-16 16:03:10,3.54,2,0
4,2,0.7315,5728.68,15.0,0.000000,1.0,BR,2,Resident Evil Operation Raccoon City Ps3,cat_6c4cfdc,0.855798,1025.0,150.0,1,NaN,N,2020-04-02 10:24:45,3.53,76,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,4,0.8191,21393.63,7.0,0.000000,1.0,BR,4,Aparelho Lipocavitação Ultrassônico + Gel Cond...,cat_a5b2091,0.808366,2009.0,434.0,1,NaN,N,2020-04-11 19:31:07,47.15,95,0
149996,4,NaN,NaN,24.0,0.384615,11.0,UY,20,"Sellos De Goma Automaticos, Personalizados.",cat_e39ab7e,0.989981,499.0,135.0,1,NaN,Y,2020-03-11 20:21:35,9.69,0,0
149997,4,NaN,NaN,3.0,0.477778,1.0,BR,19,Hélice 3 Pás Alumínio Rabeta 6.5 Hp Pesca Barc...,cat_ee6ecc8,0.763939,127.0,127.0,1,NaN,Y,2020-03-11 19:36:07,5.97,15,0
149998,4,0.6067,152906.86,1.0,0.099175,133.0,BR,3,Tela Display Lcd Galaxy J7 Neo J701 Com Brilho...,cat_237e2d0,0.382728,4373.0,123.0,1,NaN,Y,2020-03-16 07:13:24,25.83,59,0


### Renaming and formatting columns

In [33]:
df.rename(columns={"a":"a_int",
                    "p":"p_boolean","n":'n_boolean',
                     "k":"k_float","e":"e_float","h":"h_float","l":"l_float","f":"f_float","m":"m_float","m":"m_float","b":"b_float","c":"c_float","d":"d_float","o":"o_obj",
                     'i': 'produto','j':'categoria_produto','g':'pais'}
       , inplace = True)

## Transform categorical features
df['n_boolean'] = df['n_boolean'].astype("category")
df['p_boolean'] = df['p_boolean'].astype("category")


## creating columns os date and hour
df['data'] = df['fecha'].apply(lambda x: x[:10])
df['hora'] = df['fecha'].apply(lambda x: x[10:])

# Creating WoE transformation

>> Splitting the sample to contruct the WoE transformation

In [34]:
X = df.drop(['fraude'], axis=1)
y = df['fraude']

## Spliting the sample
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

## Creating the label column to save
X_train['fraude'] = y_train
X_test['fraude'] = y_test

## Saving 
X_train.to_parquet(f"{parameters.data_path}\\train_data_set.parquet", index=False)
X_test.to_parquet(f"{parameters.data_path}\\test_data_set.parquet", index=False)

## WoE transformation pipeline

In [35]:
cats_cols = utils_ml.list_subset_words( df.columns.tolist() , ["bool","cat",'obj','pais'])
numeric_cols = [x for x in df.columns.tolist() if df[x].dtype.name!="object" and df[x].dtype.name!="category" and x!="fraude" ]

features = cats_cols + numeric_cols


features

['n_boolean',
 'p_boolean',
 'categoria_produto',
 'o_obj',
 'pais',
 'a_int',
 'b_float',
 'c_float',
 'd_float',
 'e_float',
 'f_float',
 'h_float',
 'k_float',
 'l_float',
 'm_float',
 'monto',
 'score']

### Encoding

In [19]:
dff = X_train.copy()

for variable in features:

    label = "fraude"
    dtype_ = 'categorical' if dff[variable].dtype.name=='object' or dff[variable].dtype.name=='category' else 'numerical'
    monotonic = True
    save_path_pkl = '.\..\src\\features\\'

    ## creating the WoE transformation and saving in pickle object.
    dff = utils_ml.encode_woe_var(dff, variable, label, dtype_, monotonic = True, save_path_pkl = save_path_pkl )

In [20]:
features_bins_woe = utils_ml.list_subset_words( dff.columns.tolist(), ["bin"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'o_obj_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins',
 'score_bins']

In [21]:
len(features_bins_woe)

17

# Saving

In [22]:
dff

,a_int,b_float,c_float,d_float,e_float,f_float,pais,h_float,produto,categoria_produto,k_float,l_float,m_float,n_boolean,o_obj,p_boolean,fecha,monto,score,data,hora,fraude,n_boolean_bins,n_boolean_cat,n_boolean_WoE,p_boolean_bins,p_boolean_cat,p_boolean_WoE,categoria_produto_bins,categoria_produto_cat,categoria_produto_WoE,o_obj_bins,o_obj_cat,o_obj_WoE,pais_bins,pais_cat,pais_WoE,a_int_bins,a_int_cat,a_int_WoE,b_float_bins,b_float_cat,b_float_WoE,c_float_bins,c_float_cat,c_float_WoE,d_float_bins,d_float_cat,d_float_WoE,e_float_bins,e_float_cat,e_float_WoE,f_float_bins,f_float_cat,f_float_WoE,h_float_bins,h_float_cat,h_float_WoE,k_float_bins,k_float_cat,k_float_WoE,l_float_bins,l_float_cat,l_float_WoE,m_float_bins,m_float_cat,m_float_WoE,monto_bins,monto_cat,monto_WoE,score_bins,score_cat,score_WoE
101907,4,0.8020,134250.50,3.0,0.210000,6.0,BR,38,"Manta Gigante Para Sofá 2,40x1,80 Capa De Sofá...",cat_54c07aa,0.770099,1306.0,3.0,1,NaN,N,2020-03-23 10:36:21,19.00,94,2020-03-23,10:36:21,0,0,[1],-0.285787,1,['N'],0.443123,2,['cat_6a2cd5e' 'cat_12a8b9b' 'cat_e830a06' 'ca...,-0.522765,-1,Missing,0.921021,1,['BR' 'FR' 'CL' 'NZ' 'KR'],0.110792,1,"[3.50, inf)",-0.126253,3,"[0.75, 0.81)",-0.117661,1,"[636.52, inf)",-0.075286,2,"[2.50, 8.50)",0.081821,3,"[0.19, inf)",-0.184422,4,"[4.50, 6.50)",-0.151907,6,"[31.50, inf)",-0.299111,4,"[0.52, 0.79)",0.017140,4,"[1025.50, 1677.50)",-0.049991,1,"[0.50, 11.50)",0.747382,1,"[5.18, 28.29)",-0.096561,5,"[92.50, inf)",1.838788
41601,4,NaN,NaN,24.0,0.293617,3.0,BR,3,Ml-jogo Mini Bloqueio De Madeira-2 Cores -a Pa...,cat_6f55b11,0.330251,292.0,57.0,1,Y,N,2020-03-12 11:54:36,5.19,69,2020-03-12,11:54:36,0,0,[1],-0.285787,1,['N'],0.443123,1,['cat_113f0d2' 'cat_38f1214' 'cat_1a94af8' 'ca...,-1.014875,0,['Y'],0.278717,1,['BR' 'FR' 'CL' 'NZ' 'KR'],0.110792,1,"[3.50, inf)",-0.126253,-1,Missing,-0.236739,-1,Missing,-0.236739,4,"[19.50, 43.50)",-0.208394,3,"[0.19, inf)",-0.184422,3,"[2.50, 4.50)",0.049681,2,"[1.50, 5.50)",0.048033,2,"[0.21, 0.41)",-0.010287,1,"[145.50, 422.50)",0.563873,3,"[29.50, 74.50)",0.024397,1,"[5.18, 28.29)",-0.096561,1,"[65.50, 71.50)",-0.234942
57365,2,0.5120,7510.00,20.0,0.000000,387.0,AR,7,Larguero De Chasis O Mensula Renault 4 O 6 ( R...,cat_782fbcf,0.417803,5655.0,216.0,1,Y,Y,2020-03-17 19:26:05,19.01,0,2020-03-17,19:26:05,0,0,[1],-0.285787,0,['Y'],-0.554866,0,['cat_0005972' 'cat_917ba85' 'cat_9158457' ......,-4.271057,0,['Y'],0.278717,0,['AE' 'UA' 'TR' 'PY' 'PT' 'PR' 'PE' 'PA' 'NO' ...,-0.415440,0,"(-inf, 3.50)",0.553043,0,"(-inf, 0.63)",-0.253075,1,"[636.52, inf)",-0.075286,4,"[19.50, 43.50)",-0.208394,0,"(-inf, 0.07)",0.163259,10,"[159.50, inf)",-1.246162,3,"[5.50, 9.50)",-0.072438,3,"[0.41, 0.52)",0.014814,10,"[5510.50, inf)",-1.182512,5,"[153.50, 216.50)",-0.202155,1,"[5.18, 28.29)",-0.096561,0,"(-inf, 65.50)",-0.924664
61851,4,NaN,NaN,5.0,0.000000,11.0,BR,22,Alternador S10 Blazer 4.3 V6 Vortec Delco 12v ...,cat_b014e4d,0.482090,1830.0,116.0,1,NaN,N,2020-03-13 09:41:29,140.67,16,2020-03-13,09:41:29,0,0,[1],-0.285787,1,['N'],0.443123,6,['cat_ec27be3' 'cat_544bf99' 'cat_f0921fa' 'ca...,0.172424,-1,Missing,0.921021,1,['BR' 'FR' 'CL' 'NZ' 'KR'],0.110792,1,"[3.50, inf)",-0.126253,-1,Missing,-0.236739,-1,Missing,-0.236739,2,"[2.50, 8.50)",0.081821,0,"(-inf, 0.07)",0.163259,5,"[6.50, 12.50)",-0.209269,5,"[19.50, 31.50)",-0.174426,3,"[0.41, 0.52)",0.014814,5,"[1677.50, 2075.50)",-0.222775,4,"[74.50, 153.50)",-0.015681,4,"[122.76, inf)",0.783065,0,"(-inf, 65.50)",-0.924664
11189,4,0.6982,8506.70,29.0,0.655058,15.0,AR,0,Barbijo Tapa Boca Con Clip Nazal Descartable B...,cat_e600e6d,0.158487,2840.0,1107.0,1,NaN,N,2020-04-15 10:13:56,10.60,33,2020-04-15,10:13:56,0,0,[1],-0.285787,1,['N'],0.443123,1,['cat_113f0d2' 'cat_38f1214' 'cat_1a94af8' 'ca...,-1.014875,-1,Missing,0.921021,0,['AE' 'UA' 'TR' 'PY' 'PT' 'PR' 'PE' 'PA' 'NO' ...,-0.415440,1,"[3.50, inf)",-0.126253,2,"[0.69, 0.75)",-0.134229,1,"[636.52, inf)",-0.075286,4,"[19.50, 43.50)",-0.208394,3,"[0.19, inf)",-0.184422,6,"[12.50, 21.50

In [23]:
## Saving 
dff.to_parquet(f"{parameters.data_path}\\train_data_set_tranformed.parquet", index=False)

# WoE to train dataset

### Train dataset

In [26]:
@interact

def display_metrics(variable = features ):

    label = "fraude"
    dtype_ = 'categorical' if X_train[variable].dtype.name=='object' or X_train[variable].dtype.name=='category'  else 'numerical'

    display(utils_ml.WOE_IV(df = X_train , variable = variable, dtype_ = dtype_ , label = label, monotonic= True))

interactive(children=(Dropdown(description='variable', options=('n_boolean', 'p_boolean', 'categoria_produto',…

### Full dataset

In [30]:
@interact

def display_metrics(variable = features ):

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype.name=='object' or df[variable].dtype.name=='category'  else 'numerical'

    display(utils_ml.WOE_IV(df = df , variable = variable, dtype_ = dtype_ , label = label, monotonic= True))

interactive(children=(Dropdown(description='variable', options=('n_boolean', 'p_boolean', 'categoria_produto',…

# Requirements

In [96]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt